# Test Performance of ResNet152 with Confusion Matrix and ROC Curve

In [ ]:
"""
Adapted from keras example cifar10_cnn.py
Train ResNet-18 on the CIFAR10 small images dataset.

GPU run command with Theano backend
(with TensorFlow, the GPU is automatically used):

THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10.py

"""
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import data_prepare as dp
from keras.models import model_from_json

import numpy as np
import resnet
import time


lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=1, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet152_'+time.strftime("%Y%m%d_%H%M%S")+'.csv')

batch_size = 16 
nb_classes = 2
nb_epoch = 100
data_augmentation = True

# input image dimensions
img_rows, img_cols = 224, 224
# The CIFAR10 images are RGB.
img_channels = 1

def train():
    # The data, shuffled and split between train and test sets:
    (X_train, y_train), (X_test, y_test) = dp.load_Data()

    # Convert class vectors to binary class matrices.
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    # subtract mean and normalize
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_test -= mean_image
    X_train /= 128.
    X_test /= 128.
    model = resnet.ResnetBuilder.build_resnet_152((img_channels, img_rows, img_cols), nb_classes)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    #model.load_weights('resnet152_weights_tf.h5')

    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  nb_epoch=nb_epoch,
                  validation_data=(X_test, Y_test),
                  shuffle=True,
                  callbacks=[lr_reducer, csv_logger]) #, early_stopper
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)  # randomly flip images

        # Compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(X_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                            steps_per_epoch=X_train.shape[0] // batch_size,
                            validation_data=(X_test, Y_test),
                            epochs=nb_epoch, verbose=1, max_q_size=100,
                            callbacks=[lr_reducer, csv_logger]) #, early_stopper
        # predictions
        # preds = model.predict(X_test)
        # print('Predicted:', preds[0])
        
        # serialize model to JSON
        modelName = "model152_" + time.strftime("%Y%m%d_%H%M%S") + ".json"
        model_json = model.to_json()
        with open(modelName, "w") as json_file:
            json_file.write(model_json)
        print("Saved model to disk")

        # serialize weights to HDF5
        modelWeight = "model152Weight_" + time.strftime("%Y%m%d_%H%M%S") + ".h5"
        model.save_weights(modelWeight)
        print("Saved model weight to disk")
        
        return model

def predict():
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")

# model = train()
# predict()


if __name__ == '__main__':
    train()